In [1]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen():
    cap = cv2.VideoCapture(0)  # Capture video from webcam (device 0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\erick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
# this code captures video from an usb camera and cast it over LAN 
# it requires the file "index.html" inside the forlder /templates/ 

from flask import Flask, render_template, Response
import cv2
import numpy as np

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen():
    cap = cv2.VideoCapture(0)  # Capture video from webcam (device 0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') #Load face detection model

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        else:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

            # Draw rectangle around the faces
            for i, (x, y, w, h) in enumerate(faces):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"Face {i+1}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\erick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
#Encode the video into h264 format before streaming

from flask import Flask, render_template, Response
import av
import cv2
import numpy as np

fourcc_type = 'avc1'

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen():
    # Create an AVStream for output
    output = av.open('pipe:1', 'w')
    stream = output.add_stream('libx264', 20)
    stream.width = 640
    stream.height = 480
    stream.pix_fmt = 'yuv420p'

    while True:
        ret, frame = camera.read()
        if not ret:
            break
        else:
            # Convert the image from BGR color space to YUV
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV_I420)

            # Create an AVFrame and fill it with the YUV image
            av_frame = av.VideoFrame.from_ndarray(frame, format='yuv420p')
            av_frame.pts = None

            # Encode the frame
            for packet in stream.encode(av_frame):
                output.mux(packet)

            # Yield the encoded frame
            yield (b'--frame\r\n'
                   b'Content-Type: video/x-h264\r\n\r\n' + packet.to_bytes() + b'\r\n')

    # Flush the encoder and muxer
    for packet in stream.encode():
        output.mux(packet)

    output.close()

@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [11]:
# To decode H.264 video stream 
import av
import cv2
import numpy as np
import requests

def decode_stream():
    # Open the stream
    r = requests.get('http://localhost:5000/video_feed', stream=True)

    if(r.status_code == 200):
        # Create an AVContainer for input
        container = av.open(r.raw)

        # Get the video stream
        video_stream = next(s for s in container.streams if s.type == 'video')

        for packet in container.demux(video_stream):
            for frame in packet.decode():
                # Convert the frame to RGB format
                frame = frame.to_rgb().to_ndarray()

                # Display the resulting frame
                cv2.imshow('frame', frame)

                # Break the loop on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        cv2.destroyAllWindows()
    else:
        print("Received unexpected status code {}".format(r.status_code))

decode_stream()

InvalidDataError: [Errno 1094995529] Invalid data found when processing input: '<none>'

In [1]:
#Stream video using AV 
import cv2
import av
import numpy as np

# Open the camera
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Unable to read camera feed")

# Default resolutions of the frame are obtained.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create an AV stream
output = av.open('tcp://localhost:5554?listen=1', mode='w','mp4')

# Create a stream
stream = output.add_stream('mpeg4', 30)
stream.width = frame_width
stream.height = frame_height

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the image to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert the image to numpy array
    frame = np.array(frame)

    # Convert the numpy array to AVFrame
    av_frame = av.VideoFrame.from_ndarray(frame, format='rgb24')

    # Encode the frame
    packets = stream.encode(av_frame)

    # Write the packets
    for packet in packets:
        output.mux(packet)

# Close the frames
cap.release()

# Close the AV stream
output.close()

SyntaxError: positional argument follows keyword argument (3213942129.py, line 18)

In [1]:
import av

In [2]:
output = av.open('pipe:1', 'w','mp4')


In [4]:
output.close()

In [ ]:
## ZMQ TEST


In [2]:
# import the necessary packages
from imutils.video import VideoStream
import imagezmq
import argparse
import socket
import time
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-s", "--server-ip", required=True,
                help="ip address of the server to which the client will connect")
args = vars(ap.parse_args())
# initialize the ImageSender object with the socket address of the
# server
sender = imagezmq.ImageSender(connect_to="tcp://{}:5555".format(
    args["server_ip"]))
# get the host name, initialize the video stream, and allow the
# camera sensor to warmup
rpiName = socket.gethostname()
#vs = VideoStream(usePiCamera=True).start()
vs = VideoStream(src=0).start()
time.sleep(2.0)
 
while True:
    # read the frame from the camera and send it to the server
    frame = vs.read()
    sender.send_image(rpiName, frame)
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

usage: ipykernel_launcher.py [-h] -s SERVER_IP
ipykernel_launcher.py: error: the following arguments are required: -s/--server-ip


SystemExit: 2

C:\Users\erick\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
exit()

In [ ]:
# import the necessary packages
from imutils import build_montages
from datetime import datetime
import numpy as np
import imagezmq
import argparse
import imutils
import cv2
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", "--prototxt", required=True, 
                help="path to Caffe 'deploy' prototxt file")
ap.add_argument("-m", "--model", required=True,
                help="path to Caffe pre-trained model")
ap.add_argument("-c", "--confidence", type=float, default=0.2,
                help="minimum probability to filter weak detections")
ap.add_argument("-mW", "--montageW", required=True, type=int,
                help="montage frame width")
ap.add_argument("-mH", "--montageH", required=True, type=int,
                help="montage frame height")
args = vars(ap.parse_args())

# initialize the ImageHub object
imageHub = imagezmq.ImageHub()
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])
# initialize the consider set (class labels we care about and want
# to count), the object count dictionary, and the frame  dictionary
CONSIDER = set(["dog", "person", "car"])
objCount = {obj: 0 for obj in CONSIDER}
frameDict = {}
# initialize the dictionary which will contain  information regarding
# when a device was last active, then store the last time the check
# was made was now
lastActive = {}
lastActiveCheck = datetime.now()
# stores the estimated number of Pis, active checking period, and
# calculates the duration seconds to wait before making a check to
# see if a device was active
ESTIMATED_NUM_PIS = 4
ACTIVE_CHECK_PERIOD = 10
ACTIVE_CHECK_SECONDS = ESTIMATED_NUM_PIS * ACTIVE_CHECK_PERIOD
# assign montage width and height so we can view all incoming frames
# in a single "dashboard"
mW = args["montageW"]
mH = args["montageH"]
print("[INFO] detecting: {}...".format(", ".join(obj for obj in CONSIDER)))

# start looping over all the frames
while True:
    # receive RPi name and frame from the RPi and acknowledge
    # the receipt
    (rpiName, frame) = imageHub.recv_image()
    imageHub.send_reply(b'OK')
    # if a device is not in the last active dictionary then it means
    # that its a newly connected device
    if rpiName not in lastActive.keys():
        print("[INFO] receiving data from {}...".format(rpiName))
    # record the last active time for the device from which we just
    # received a frame
    lastActive[rpiName] = datetime.now()
    
    # resize the frame to have a maximum width of 400 pixels, then
    # grab the frame dimensions and construct a blob
    frame = imutils.resize(frame, width=400)
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 
                                 0.007843, (300, 300), 127.5)
    # pass the blob through the network and obtain the detections and
    # predictions
    net.setInput(blob)
    detections = net.forward()
    # reset the object count for each object in the CONSIDER set
    objCount = {obj: 0 for obj in CONSIDER}
    
    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > args["confidence"]:
            # extract the index of the class label from the
            # detections
            idx = int(detections[0, 0, i, 1])
            # check to see if the predicted class is in the set of
            # classes that need to be considered
            if CLASSES[idx] in CONSIDER:
                # increment the count of the particular object
                # detected in the frame
                objCount[CLASSES[idx]] += 1
                # compute the (x, y)-coordinates of the bounding box
                # for the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                # draw the bounding box around the detected object on
                # the frame
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                              (255, 0, 0), 2)
                
                                # draw the sending device name on the frame
                cv2.putText(frame, rpiName, (10, 25),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                # draw the object count on the frame
                label = ", ".join("{}: {}".format(obj, count) for (obj, count) in
                                  objCount.items())
                cv2.putText(frame, label, (10, h - 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255,0), 2)
                # update the new frame in the frame dictionary
                frameDict[rpiName] = frame
                # build a montage using images in the frame dictionary
                montages = build_montages(frameDict.values(), (w, h), (mW, mH))
                # display the montage(s) on the screen
                for (i, montage) in enumerate(montages):
                    cv2.imshow("Home pet location monitor ({})".format(i), 
                               montage)
                # detect any kepresses
                key = cv2.waitKey(1) & 0xFF

In [ ]:
# run this program on each RPi to send a labelled image stream
import socket
import time
from imutils.video import VideoStream
import imagezmq

sender = imagezmq.ImageSender(connect_to='tcp://localhost:5555')

rpi_name = socket.gethostname() # send RPi hostname with each image
print(rpi_name)
picam = VideoStream(src=0).start()
time.sleep(2.0)  # allow camera sensor to warm up
while True:  # send images as stream until Ctrl-C
    image = picam.read()
    sender.send_image(rpi_name, image)

In [ ]:
#OpenCV client 

import cv2
import imagezmq
image_hub = imagezmq.ImageHub()
while True:  # show streamed images until Ctrl-C
    rpi_name, image = image_hub.recv_image()
    cv2.imshow(rpi_name, image) # 1 window for each RPi
    cv2.waitKey(1)
    image_hub.send_reply(b'OK')

In [ ]:
import cv2
import imagezmq

# Initialize the ImageHub object
image_hub = imagezmq.ImageHub()

while True:
    # Receive a frame from the client
    client_name, frame = image_hub.recv_image()

    # Display the frame
    cv2.imshow(client_name, frame)

    # If the 'q' key is pressed, break from the loop
    if cv2.waitKey(1) == ord('q'):
        break

    # Send a reply to the client (necessary to continue the loop on the client side)
    image_hub.send_reply(b'OK')

# After the loop, close the display window
cv2.destroyAllWindows()





In [ ]:
import cv2
import imagezmq
import socket
import signal
import time   # For the demo only

def signal_handler(signal, frame):
    global interrupted
    interrupted = True

signal.signal(signal.SIGINT, signal_handler)

# Initialize the ImageSender object
sender = imagezmq.ImageSender(connect_to='tcp://localhost:5555')

# Assign a name to the client
client_name = socket.gethostname() # send RPi hostname with each image
print(client_name)

# Initialize a VideoCapture object
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the VideoCapture object
    ret, frame = cap.read()

    # If a frame was successfully read...
    if ret:
        # Send the frame to the server
        sender.send_image(client_name, frame)
    
    if interrupted:
        print("Gotta go")
        break

In [ ]:
## BARD VERSION WEBSOCKET TRANSMITIONS

import cv2
import socket
import time
import threading

# Define the IP address and port for broadcasting
HOST = 'localhost'  # Replace with the receiving device's IP address
PORT = 8000

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 for the first webcam

# Set the frame rate to 30 frames per second
cap.set(cv2.CAP_PROP_FPS, 30)

# Define the output video file name
output_filename = 'live_video.mp4'

# Create a socket object for sending data over TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Start the server and listen for incoming connections
sock.bind((HOST, PORT))
sock.listen(5)
print("Server started and listening...")

# Create a thread to handle streaming to each client
def handle_client(client_socket):
    global cap, video_writer
    try:
        # Start the video writer if it hasn't been started already
        if not video_writer:
            video_writer = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'MP4V'), 30, (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

        # Continuously capture frames from the webcam and send them to the client
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the frame to a NumPy array
            data = frame.tobytes()

            # Send the frame data to the client
            client_socket.sendall(data)

            # Write the frame to the MP4 file if it is greater than a specific size
            if len(data) > 1024 * 1024:
                # Write the current frame to the MP4 file
                video_writer.write(frame)

    except:
        client_socket.close()

# Accept incoming connections from clients
while True:
    try:
        client_socket, address = sock.accept()
        print(f"Connected to client: {address}")

        # Create a new thread to handle streaming to this client
        thread = threading.Thread(target=handle_client, args=(client_socket,))
        thread.start()

    except:
        pass

# Close the video capture object and video writer
cap.release()
if video_writer:
    video_writer.release()

# Close the socket object
sock.close()

# Close the display window
cv2.destroyAllWindows()

In [ ]:
import cv2
import socket
import threading
import time
import base64
from http.server import BaseHTTPRequestHandler, HTTPServer

# Define the IP address and port for broadcasting
HOST = '192.168.0.116'  # Replace with the receiving device's IP address
PORT = 8228

# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 for the first webcam

# Set the frame rate to 30 frames per second
cap.set(cv2.CAP_PROP_FPS, 30)

# Define the output video file name
output_filename = 'live_video.mp4'

# Class for handling HTTP requests
class HTTPHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        # Check if the requested path is the video stream
        if self.path == '/live_video.mp4':
            # Read the current frame from the webcam
            ret, frame = cap.read()

            # Convert the frame to a NumPy array
            frame_data = frame.tobytes()

            # Encode the frame data as base64
            encoded_data = base64.b64encode(frame_data)

            # Send the encoded frame data
            self.send_response(200)
            self.send_header('Content-Type', 'video/mp4')
            self.end_headers()
            self.wfile.write(encoded_data)
            return

        # Send error response if the requested path is invalid
        self.send_error(404)
        self.end_headers()


# Create a TCP socket server for HTTP communication
server = HTTPServer(('localhost', PORT), HTTPHandler)
server.allow_reuse_address = True
print("Starting HTTP server...")
server.serve_forever()


# Start the HTTP server
server_thread = threading.Thread(target=server.serve_forever)
server_thread.daemon = True
server_thread.start()

# Continuously capture frames from the webcam and send them as HTTP requests
while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()

    # Convert the frame to a NumPy array
    frame_data = frame.tobytes()

    # Encode the frame data as base64
    encoded_data = base64.b64encode(frame_data)

    # Send the encoded frame data as an HTTP request
    req = "GET /live_video.mp4 HTTP/1.1\r\nHost: localhost\r\nConnection: close\r\nContent-Length: {}\r\n\r\n{}".format(len(encoded_data), encoded_data.decode())
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((HOST, PORT))
    sock.sendall(req.encode())
    sock.close()

    # Write the frame to the MP4 file if it is greater than a specific size
    if len(frame_data) > 1024 * 1024:
        # Write the current frame to the MP4 file
        video_writer = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'MP4V'), 30, (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        video_writer.write(frame)

# Close the video capture object and video writer
cap.release()
if video_writer:
    video_writer.release()

# Close the server thread
server.server_close()

Starting HTTP server...


127.0.0.1 - - [15/Dec/2023 18:34:26] "GET /live_video.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2023 18:34:26] "GET /live_video.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2023 18:37:41] "GET /live_video.mp4 HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2023 18:37:41] "GET /live_video.mp4 HTTP/1.1" 200 -


In [ ]:
import requests
import cv2
import base64

# Define the IP address and port of the Flask application
HOST = '192.168.1.255'  # Replace with the Flask application's IP address
PORT = 8080

# Create a request object for the live video stream
url = f'http://{HOST}:{PORT}/live_video.mp4'

# Continuously fetch the live video stream and decode the base64 data
while True:
    # Send a GET request for the live video stream
    response = requests.get(url).content

    # Decode the base64 data into a NumPy array
    decoded_data = base64.b64decode(response)

    # Convert the NumPy array to a frame
    frame = cv2.imdecode(np.frombuffer(decoded_data, dtype='uint8'), -1)

    # Display the frame
    cv2.imshow('Received Video', frame)

    # Check for the 'q' key to quit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Close the video stream
cv2.destroyAllWindows()

In [2]:
from imutils.video import VideoStream

In [1]:


from flask import Flask, render_template, Response
import requests
import cv2
import numpy as np


def gen():
    #cap = cv2.VideoCapture(0)  # Capture video from webcam (device 0)
    #ret, imgOriginalScene = cap.read()
    url = "http://192.168.0.108:5000/video_feed"
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    imgOriginalScene = cv2.imdecode(img_arr, -1)
    cv2.imshow("IPcamera", imgOriginalScene)
    cv2.namedWindow('IPcamera', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('IPcamera', 300, 300)
    

    


if __name__ == '__main__':
    gen()

KeyboardInterrupt: 

In [ ]:
#MQTT IMAGE TRANSMISSION
import cv2
import threading
import time
import base64
from paho.mqtt import client as mqtt_client
import random


# Define the IP address and port for broadcasting
broker = '192.168.0.108'  # Broadcast to all devices on the network
port = 1883
topic = "test"


# Generate a Client ID with the publish prefix.
client_id = f'ImageProvider-{random.randint(0, 1000)}'
# username = 'emqx'
# password = 'public'

################################################################################
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    # client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client


def publish(client, msg):
    msg_count = 1
    #print(msg)
    result = client.publish(topic, msg) # result: [0, 1]
    status = result[0]
    if status == 0:
        msg_count += 1
        #print("Message Printed")
    else:
        print(f"Failed to send message to topic {topic}")
    
    
##################################################################################


client = connect_mqtt()
client.loop_start()
# Initialize the video capture object
cap = cv2.VideoCapture(0)  # 0 for the first webcam
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') #Load face detection model
# Set the frame rate to 30 frames per second
cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(3, 1280)
cap.set(4, 480)
cont =1
while cont == 1:
    #cont =2
    # Read the current frame from the webcam
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))
    
    # Draw rectangle around the faces
    for i, (x, y, w, h) in enumerate(faces):
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"Face {i+1}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    
    resize= cv2.resize(frame, (1280, 480))
    frame = cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
    #print(frame.shape)
    res, frame = cv2.imencode('.jpg', frame)    # from image to binary buffer
    
    #print(frame)
    
    if not ret:
        break

    # Convert the frame to a Base64 array
    frame = base64.b64encode(frame) #frame.tobytes()
    
    # Send the encoded frame data as an Mqtt message
    publish(client, frame)

# Close the video capture object and video writer
client.loop_stop()
cap.release()

###################################################################################


In [1]:
import cv2
index = 0
arr = []
while True:
    cap = cv2.VideoCapture(index)
    if not cap.read()[0]:
        break
    else:
        arr.append(index)
    cap.release()
    index += 1
print( arr)

[0, 1]
